In [ ]:
import os, psutil
import glob
import collections
import pprint
import tinydb as tdb
import json
import subprocess

In [108]:
## new imports
from functools import reduce

In [5]:
def get_benchmark():
    ior_path='/home/neeraj/ior/src/'
    rand_command = ior_path+'ior -a=POSIX -z -O summaryFormat=JSON'
    seq_command = ior_path+'ior -a=POSIX -O summaryFormat=JSON'
    sequential = json.loads(subprocess.check_output(rand_command, shell=True).decode('utf-8'))
    random = json.loads(subprocess.check_output(seq_command, shell=True).decode('utf-8'))
    filesystem  = subprocess.check_output("df -P .", shell=True).decode('utf-8')
    filesystem=filesystem[filesystem.find('\n'):].split()[0]
    read=1
    write=0
    return {
        'filesystem': filesystem,
        sequential['tests'][0]['Options']['ordering in a file'] :
        {
            sequential['summary'][read]['operation']:{
            'bwMaxMIB' :    sequential['summary'][read]['bwMaxMIB'],
            'bwMinMIB' :    sequential['summary'][read]['bwMinMIB'],
            'bwMeanMIB' :    sequential['summary'][read]['bwMeanMIB'],
            'openTime' :    sequential['tests'][0]['Results'][0][read]['openTime'],
            'wrRdTime' :    sequential['tests'][0]['Results'][0][read]['wrRdTime'],
            'totalTime' :    sequential['tests'][0]['Results'][0][read]['totalTime']
            },
             sequential['summary'][write]['operation']:{
            'bwMaxMIB' :    sequential['summary'][write]['bwMaxMIB'],
            'bwMinMIB' :    sequential['summary'][write]['bwMinMIB'],
            'bwMeanMIB' :    sequential['summary'][write]['bwMeanMIB'],
            'openTime' :    sequential['tests'][0]['Results'][0][write]['openTime'],
            'wrRdTime' :    sequential['tests'][0]['Results'][0][write]['wrRdTime'],
            'totalTime' :    sequential['tests'][0]['Results'][0][write]['totalTime']
            }
        },
                random['tests'][0]['Options']['ordering in a file'] :
        {
            random['summary'][read]['operation']:{
            'bwMaxMIB' :    random['summary'][read]['bwMaxMIB'],
            'bwMinMIB' :    random['summary'][read]['bwMinMIB'],
            'bwMeanMIB' :    random['summary'][read]['bwMeanMIB'],
            'openTime' :    random['tests'][0]['Results'][0][read]['openTime'],
            'wrRdTime' :    random['tests'][0]['Results'][0][read]['wrRdTime'],
            'totalTime' :    random['tests'][0]['Results'][0][read]['totalTime']
            },
             random['summary'][write]['operation']:{
            'bwMaxMIB' :    random['summary'][write]['bwMaxMIB'],
            'bwMinMIB' :    random['summary'][write]['bwMinMIB'],
            'bwMeanMIB' :    random['summary'][write]['bwMeanMIB'],
            'openTime' :    random['tests'][0]['Results'][0][write]['openTime'],
            'wrRdTime' :    random['tests'][0]['Results'][0][write]['wrRdTime'],
            'totalTime' :    random['tests'][0]['Results'][0][write]['totalTime']
            }
        }
    }
# sample =get_benchmark()
# print(json.dumps(sample, sort_keys=True, indent=4))


In [ ]:
def bytes_converter(b, unit='GB'):
    """
    Returns the b bytes in the specified unit
    parms: b <float>, unit <str> default='GB'
    Notes: to extend prev_num+1 if unit is 'XB' or unit is 'xb' else None  <- replace None with, to extend bytes_converter as needed
    """
    multiplier = (1 if unit is 'KB'or unit is 'kb' else 2 if unit is 'MB' or unit is 'mb' else 3 if unit is 'GB' or unit is 'gb' else None)
    return b/(1024**multiplier)

def get_mounted_partitions(unit='GB'):

    res_d=collections.defaultdict(list)
    for dev,mount_point,fs,opts in psutil.disk_partitions():
        res = {}
        a = psutil.disk_usage(path=mount_point)
#         print(mount_point)
        res['dev'] = dev
        res['mount_point'] = mount_point
        res['fs'] = fs
        res['opts']=opts
        res['memory_details_'+unit]={label_me(i):bytes_converter(j,unit) if j != a[-1] else j  for i, j in enumerate(a)}
        res_d[dev.split('/')[-1].rstrip('123456789')].append(res)
    return res_d

def label_me(x):
    return 'total' if x == 0 else 'used' if x == 1 else 'free' if x ==2 else 'percent' if x ==3 else 'Unknown'

def get_RAM(unit='GB'):
    a=psutil.virtual_memory()
    res = {'total_'+unit:bytes_converter(a[0],unit),
           'available_'+unit:bytes_converter(a[1],unit),
           'percent':a[2],
           'used_'+unit:bytes_converter(a[3],unit),
           'free_'+unit:bytes_converter(a[4],unit),
           'active_'+unit:bytes_converter(a[5],unit),
           'inactive_'+unit:bytes_converter(a[6],unit),
           'buffers_'+unit:bytes_converter(a[7],unit),
           'cached_'+unit:bytes_converter(a[8],unit),
           'shared_'+unit:bytes_converter(a[9],unit),
           'slab_'+unit:bytes_converter(a[10],unit),
          }
    return res

def get_swap_memory(unit='GB'):
    a=psutil.swap_memory()
    swap_res={}
    swap_res['total_'+unit]=bytes_converter(a[0],unit=unit)
    swap_res['used_'+unit]=bytes_converter(a[1],unit=unit)
    swap_res['free_'+unit]=bytes_converter(a[2],unit=unit)
    swap_res['percent_'+unit]=a[3]
    swap_res['sin_'+unit]=bytes_converter(a[4],unit=unit)
    swap_res['sout_'+unit]=bytes_converter(a[5],unit=unit)
    return swap_res

In [52]:
def bash_jason(cmd):
    return json.loads(subprocess.check_output(cmd, shell=True).decode('utf-8'))
    

In [47]:
def get_block_info():
    cmd = 'lsblk -JS -io name,model,rota,WWN,HCTL,vendor,hotplug,subsystems,tran,min-io,opt-io,sched'
    blocks = json.loads(subprocess.check_output(cmd, shell=True).decode('utf-8'))
    c_block=collections.defaultdict(list)
    for idx, block in enumerate(blocks['blockdevices']):
        t_d={}
        for i,j in block.items():
            if i == 'rota':
                t_d['disk_type']= 'SSD' if j == False else 'HDD'
            elif i == 'wwn':
                t_d['unique_strorage_id']=j
            elif i =='hctl':
                t_d['Host:Channel:Target:Lun']= j
            else:
                t_d[i]=j
        c_block[blocks['blockdevices'][idx]['name']].append(t_d)
    return c_block
    
    

In [46]:
def print_hashlines(n):
    print('#'*n)
    
def main():
    print_hashlines(80)
    print("Block Information")
    print_hashlines(80)
    pprint.pprint(get_block_info())

    print_hashlines(80)
    print("Mounted Partition Information")
    print_hashlines(80)
    pprint.pprint(get_mounted_partitions())

    print_hashlines(80)
    print("RAM and Swap Information")
    print_hashlines(80)
    swap_ram={
        "swap":get_swap_memory(),
        "ram":get_RAM(),
        }
    pprint.pprint(swap_ram)
    
    print_hashlines(80)
    print("IOR Benchmarks")
    print_hashlines(80)
    pprint.pprint(get_benchmark())

In [49]:
if __name__ == '__main__':
    main()


################################################################################
Block Information
################################################################################
defaultdict(<class 'list'>,
            {'sda': [{'Host:Channel:Target:Lun': '0:0:0:0',
                      'disk_type': 'SSD',
                      'hotplug': False,
                      'min-io': 512,
                      'model': 'SAMSUNG_MZNTE512HMJH-00000',
                      'name': 'sda',
                      'opt-io': 0,
                      'sched': 'cfq',
                      'subsystems': 'block:scsi:pci',
                      'tran': 'sata',
                      'unique_strorage_id': '0x500253880045cb51',
                      'vendor': 'ATA     '}]})
################################################################################
Mounted Partition Information
################################################################################
defaultdict(<class 'list'>,
            {'luk

In [48]:
get_block_info()

defaultdict(list,
            {'sda': [{'name': 'sda',
               'model': 'SAMSUNG_MZNTE512HMJH-00000',
               'disk_type': 'SSD',
               'unique_strorage_id': '0x500253880045cb51',
               'Host:Channel:Target:Lun': '0:0:0:0',
               'vendor': 'ATA     ',
               'hotplug': False,
               'subsystems': 'block:scsi:pci',
               'tran': 'sata',
               'min-io': 512,
               'opt-io': 0,
               'sched': 'cfq'}]})

In [225]:
%%time
mounts = bash_jason("lsblk -J -io name,mountpoint")

CPU times: user 1.49 ms, sys: 1.51 ms, total: 3 ms
Wall time: 8.49 ms


In [288]:
%%time
def mount_lookup_child(child):
    for children in child:
        if 'children' in children.keys():
            for x in child_flow(children['children']):
                yield (x)
        yield (children['name'],children['mountpoint'])

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.01 µs


In [387]:
%%time
mnt = list(filter(lambda x : x[1] is not None ,mount_lookup_child(mounts['blockdevices'])))
fio_rw_ops= ['read', 'write', 'randread','randwrite','rw', 'readwrite', 'randrw']
fio_size_ops = ['4K', '8K', '16K', '32K', '1M', '4M', '8M', '16M' ]


NameError: name 'rw' is not defined

In [365]:
def all_checker(inp):
    return True if len(inp.split(',')) == 1 and inp[0] == '0' else False
    

In [378]:
def gimme_choice(mnt,prompt):
    print(prompt)
    for i,j in enumerate(mnt):
        print(i+1,j)

    while True:
        try:
            inp=input('Select index (comma separated) [0 : all]:')
            if all_checker(inp):
                selects= list(range(len(mnt)))
            else:
                selects=list(map(lambda x: int(x)-1,inp.split(',')))
                for i in selects:
                    if i > len(mnt) or i < 0:
                        raise ValueError('Invalid index value')
            break
        except ValueError as e:
            print(e)
            pass
    return selects

In [393]:
def select2fio_ops(select, ops):
    for i in select:
        yield ops[i] if type(ops[i]) != tuple else ops[i][1] 

In [396]:
selects_mounts=gimme_choice(mnt,"Index, (name, mount point)")
selects_rw=gimme_choice(fio_rw_ops, "Index, RW_OPTs")
selects_size=gimme_choice(fio_size_ops,"Index, Size opts")

Index, (name, mount point)
1 ('sda2', '/boot/efi')
2 ('lukssda3', '/home')
3 ('sda6', '/')
4 ('zram0', '[SWAP]')
Select index (comma separated) [0 : all]:1,2,3
Index, RW_OPTs
1 read
2 write
3 randread
4 randwrite
5 rw
6 readwrite
7 randrw
Select index (comma separated) [0 : all]:0
Index, Size opts
1 4K
2 8K
3 16K
4 32K
5 1M
6 4M
7 8M
8 16M
Select index (comma separated) [0 : all]:0


In [401]:
mount_points=list(select2fio_ops(selects_mounts,mnt))
rw_ops=list(select2fio_ops(selects_rw,fio_rw_ops))
size_ops=list(select2fio_ops(selects_size,fio_size_ops))

In [452]:
for dire in mount_points:
    for rw in rw_ops:
        for size in size_ops:
            print(f"fio --output-format=json  --name=global --rw={rw} --size={size} --name=job1 --directory={dire} --direct=1 --fsync=1")

fio --output-format=json  --name=global --rw=read --size=4K --name=job1 --directory=/boot/efi --direct=1 --fsync=1
fio --output-format=json  --name=global --rw=read --size=8K --name=job1 --directory=/boot/efi --direct=1 --fsync=1
fio --output-format=json  --name=global --rw=read --size=16K --name=job1 --directory=/boot/efi --direct=1 --fsync=1
fio --output-format=json  --name=global --rw=read --size=32K --name=job1 --directory=/boot/efi --direct=1 --fsync=1
fio --output-format=json  --name=global --rw=read --size=1M --name=job1 --directory=/boot/efi --direct=1 --fsync=1
fio --output-format=json  --name=global --rw=read --size=4M --name=job1 --directory=/boot/efi --direct=1 --fsync=1
fio --output-format=json  --name=global --rw=read --size=8M --name=job1 --directory=/boot/efi --direct=1 --fsync=1
fio --output-format=json  --name=global --rw=read --size=16M --name=job1 --directory=/boot/efi --direct=1 --fsync=1
fio --output-format=json  --name=global --rw=write --size=4K --name=job1 --di

In [444]:
import configparser

In [445]:
conf=configparser.ConfigParser()

In [447]:
conf.read('/home/neeraj/Sync/Assignments_coursework/IIT_Assignments/AOS/git_repo/Probing/probe_config.config')

['/home/neeraj/Sync/Assignments_coursework/IIT_Assignments/AOS/git_repo/Probing/probe_config.config']

In [448]:
conf.sections()

[]

In [449]:
i=conf.defaults().items()

In [450]:
k=dict(i)['fio_install']

In [455]:
bash_jason(k+'/fio --output-format=json --name=global --rw=read --size=4K --name=job1 --directory=/home/neeraj --direct=1 --fsync=1 ')

{'fio version': 'fio-3.13',
 'timestamp': 1553426906,
 'timestamp_ms': 1553426906516,
 'time': 'Sun Mar 24 06:28:26 2019',
 'global options': {'name': 'global', 'rw': 'read', 'size': '4K'},
 'jobs': [{'jobname': 'job1',
   'groupid': 0,
   'error': 0,
   'eta': 2147483647,
   'elapsed': 1,
   'job options': {'name': 'job1',
    'directory': '/home/neeraj',
    'direct': '1',
    'fsync': '1'},
   'read': {'io_bytes': 4096,
    'io_kbytes': 4,
    'bw_bytes': 1365333,
    'bw': 1333,
    'iops': 333.333333,
    'runtime': 3,
    'total_ios': 1,
    'short_ios': 0,
    'drop_ios': 0,
    'slat_ns': {'min': 0, 'max': 0, 'mean': 0.0, 'stddev': 0.0},
    'clat_ns': {'min': 2871259,
     'max': 2871259,
     'mean': 2871259.0,
     'stddev': 0.0,
     'percentile': {'1.000000': 2867200,
      '5.000000': 2867200,
      '10.000000': 2867200,
      '20.000000': 2867200,
      '30.000000': 2867200,
      '40.000000': 2867200,
      '50.000000': 2867200,
      '60.000000': 2867200,
      '70.000